In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Adjust path if the package is not installed
import sys
sys.path.append('.')  # Assumes root contains battery_ocv_toolbox/
# Import the package
from ocv_teaching import OCV, ActiveMaterial, OCVBlending, CellOCVReconstruction
from ocv_teaching.plot_ocv import plot_ocv
from ocv_teaching.utils import interpolate, new_plot

In [2]:
NCM811_df = pd.read_csv('../ocv_data/NMC811_half_cell_ocv.csv')
Graphite_df = pd.read_csv('../ocv_data/Graphite_half_cell_ocv.csv')

In [3]:
NCM811_ocv = OCV(NCM811_df["SOC"], NCM811_df["Voltage"], NCM811_df["Voltage"])
Graphite_ocv = OCV(Graphite_df["SOC"], Graphite_df["Voltage"], Graphite_df["Voltage"])
NCM811_mat = ActiveMaterial(NCM811_ocv, specific_capacity=212, formation_loss=0.09)
Graphite_mat = ActiveMaterial(Graphite_ocv, specific_capacity=372, formation_loss=0.08)


Plot OCVs vs capacity normalized

In [4]:

from ipywidgets import interact, FloatSlider
def plot_np_ratio(np_ratio,v_min,v_max):
    cell = CellOCVReconstruction(NCM811_mat, Graphite_mat, np_ratio=np_ratio, v_min=v_min, v_max=v_max)

    # Reconstruct voltage of the cell over the full range
    an0_full = cell.align_anode_cathode(1)
    an1_full = cell.align_anode_cathode(0)
    volt_cell_full  = cell.reconstruct_voltage(
        an0_full, 1, an1_full, 0, direction="discharge"
    )[0]
    # Get stoichiometeries within voltage limits
    an0, cath0, an1, cath1 = cell.get_stoichiometries()
    sol = np.linspace(0,1,100)
    # Relate anode to cathode stoichiometry
    sol_an_plot = (sol-an0_full)/(an1_full-an0_full)
    volt_cath = interpolate(cell.cath.ocv.soc, cell.cath.ocv.get_voltage("charge"), sol)
    volt_an = interpolate(cell.an.ocv.soc, cell.an.ocv.get_voltage("charge"), sol)

    # Plot the results
    battery_color={"gray": "#333333",
                       "bright_gray": "#E7E6E6",
                        "gray_blue": "#44546A",
                        "navy_blue": "#003B73",
                        "light_blue":"#5DADE2",
                        "orange": "#F39C12",
                        "green": "#70AD47"}
    fig, ax = new_plot(figsize=(8, 6))
    ax.set_xlim(0,1.2)
    ax.set_ylim(0,4.5)
    ax.set_ylabel("Voltage [V]")
    ax.set_xlabel("Normalized Capacity [Ah]")
    ax.grid()
    ax.plot(sol_an_plot , volt_an, label="Graphite OCV",color=battery_color["light_blue"])
    ax.plot(1-sol, volt_cath, label="NCM811 OCV",color=battery_color["green"])
    ax.plot(sol,volt_cell_full, label="Cell OCV", color=battery_color["navy_blue"])
    ax.hlines([v_min, v_max], 0, 1.2, linestyles='dashed', colors='black')
    ax.plot([1-cath0,1-cath1],[v_min,v_max],marker="o", color=battery_color["navy_blue"],linestyle='None')
    ax.legend(loc='right')

    # ## Draw the usage span
    # # compute the x‑coordinates of the usage span
    start = 1 - cath0
    end   = 1 - cath1
    mid   = 0.5*(start + end)
    usage = cath0 - cath1

    # draw a double‑headed arrow at y=3
    ax.annotate(
        '',                          # no text
        xy=(end, 3),                 # arrow head at right
        xytext=(start, 3),           # arrow tail at left
        arrowprops=dict(
            arrowstyle='<->',        # double‑headed
            color='black',
            lw=2
        )
    )

    # label it just above the arrow
    ax.text(
        mid, 3.05,                   # position (slightly above y=3)
        f'Cathode usage = {usage:.3f}',
        ha='center', va='bottom',
        color='black'
    )
    plt.show()

interact(
    plot_np_ratio,
    np_ratio=FloatSlider(min=0.9, max=1.2, step=0.01, value=1.1, description="N:P Ratio"),
    v_min=FloatSlider(min=2, max=3.6, step=0.01, value=2.8, description="Lower voltage limit"),
    v_max=FloatSlider(min=3.6, max=4.2, step=0.01, value=4.2, description="Upper voltage limit"),
)

interactive(children=(FloatSlider(value=1.1, description='N:P Ratio', max=1.2, min=0.9, step=0.01), FloatSlide…

<function __main__.plot_np_ratio(np_ratio, v_min, v_max)>

In [ ]:
from ocv_teaching import utils

def plot_aging_modes(LAMPE,LAMNE,LLI):
    cell = CellOCVReconstruction(NCM811_mat, Graphite_mat, np_ratio=1.1, v_min=2.8, v_max=4.2)

    # Reconstruct voltage of the cell over the full range
    an0_full = cell.align_anode_cathode(1)
    an1_full = cell.align_anode_cathode(0)
    volt_cell_full  = cell.reconstruct_voltage(
        an0_full, 1, an1_full, 0, direction="discharge"
    )[0]
    # Get stoichiometeries within voltage limits
    an0, cath0, an1, cath1 = cell.get_stoichiometries()
    sol = np.linspace(0,1,100)
    # Relate anode to cathode stoichiometry
    sol_an_plot = (sol-an0_full)/(an1_full-an0_full)
    volt_cath = interpolate(cell.cath.ocv.soc, cell.cath.ocv.get_voltage("charge"), sol)
    volt_an = interpolate(cell.an.ocv.soc, cell.an.ocv.get_voltage("charge"), sol)
    volt_cell = cell.reconstruct_voltage(an0, cath0, an1, cath1, direction="charge")[0]
    soc_cell = (sol-cath1)/(cath0-cath1)#Cut soc cell at sol_an and sol_cath limits
    soc_cell_clipped = np.clip(soc_cell, sol_an_plot[0], sol_an_plot[-1])
    soc_cell_clipped = np.clip(soc_cell_clipped, sol[0], sol[-1])
    # Cut voltages at the same limits
    v_min = utils.interpolate(soc_cell,volt_cell,soc_cell_clipped[0])
    v_max = utils.interpolate(soc_cell,volt_cell,soc_cell_clipped[-1])
    volt_cell_clipped = np.clip(volt_cell, v_min, v_max)


    ## Aged curves
    # Apply modes
    np_aged = cell.np_ratio*(1-LAMNE)
    np_offset_aged = 1-(1-cell.np_offset)*(1-LLI)
    # Align anode
    an0 = cell.align_anode_cathode(1,np_aged, np_offset_aged)
    an1 = cell.align_anode_cathode(0,np_aged, np_offset_aged)
    sol_an_aged = (sol-an0)/(an1-an0)
    sol_cath_aged = (1-sol*(1-LAMPE))
    # Align cell
    an0_aged, cath0_aged, an1_aged, cath1_aged = cell.get_stoichiometries(np_offset=np_offset_aged, np_ratio=np_aged)

    # Correct for values below 0 or above 1
    volt_cell_aged = cell.reconstruct_voltage(an0_aged, cath0_aged, an1_aged, cath1_aged,direction="charge")[0]
    soc_cell_aged = (sol-cath1_aged)/(cath0_aged-cath1_aged)
    #Cut soc cell at sol_an and sol_cath limits
    soc_cell_aged_clipped = np.clip(soc_cell_aged, sol_an_aged[0], sol_an_aged[-1])
    soc_cell_aged_clipped = np.clip(soc_cell_aged_clipped, sol_cath_aged[-1], sol_cath_aged[0])
    # Cut voltages at the same limits
    v_min = utils.interpolate(soc_cell_aged,volt_cell_aged,soc_cell_aged_clipped[0])
    v_max = utils.interpolate(soc_cell_aged,volt_cell_aged,soc_cell_aged_clipped[-1])
    volt_cell_aged_clipped = np.clip(volt_cell_aged, v_min, v_max)

    # Plot the results
    battery_color={"gray": "#333333",
                        "bright_gray": "#E7E6E6",
                        "gray_blue": "#44546A",
                        "navy_blue": "#003B73",
                        "light_blue":"#5DADE2",
                        "orange": "#F39C12",
                        "green": "#70AD47"}
    fig, ax = new_plot(figsize=(8, 6))
    ax.set_xlim(0,1.2)
    ax.set_ylim(0,4.5)
    ax.set_ylabel("Voltage [V]")
    ax.set_xlabel("Normalized Capacity [Ah]")
    ax.grid()
    ax.plot(sol_an_plot , volt_an,color=battery_color["light_blue"], linestyle='--')
    ax.plot(1-sol, volt_cath,color=battery_color["green"], linestyle='--')
    ax.plot(soc_cell_clipped,volt_cell_clipped, color=battery_color["navy_blue"], linestyle='--')
    # Aged curves
    ax.plot(sol_an_aged , volt_an, label="Graphite OCV", color=battery_color["light_blue"])
    ax.plot(sol_cath_aged, volt_cath, label="NCM811 OCV", color=battery_color["green"])
    ax.plot(soc_cell_aged_clipped,volt_cell_aged_clipped, label="Cell OCV", color=battery_color["navy_blue"])
    ax.legend(loc='lower right')


    ## SoH Arrows    
    # existing initial arrow
    start = soc_cell_clipped[0]  # start in SOC‑space
    end   = soc_cell_clipped[-1] # end in SOC‑space
    mid   = 0.5*(start + end)
    usage = end - start

    ax.annotate(
        '',                      
        xy=(end, 3),             
        xytext=(start, 3),       
        arrowprops=dict(
            arrowstyle='<->',    
            color='black',
            linewidth=2,
            linestyle='--'       # use dashed line for arrow
        )
    )

    # — now add the aged‑cell arrow at y=2 — #

    # aged arrow start/end in SOC‑space
    start_aged = soc_cell_aged_clipped[0]
    end_aged   = soc_cell_aged_clipped[-1]
    mid_aged   = 0.5*(start_aged + end_aged)
    usage_aged = end_aged - start_aged

    # compute SoH = (aged span) / (fresh span)
    soh = usage_aged / usage

    ax.annotate(
        '',
        xy=(end_aged, 2),        
        xytext=(start_aged, 2),  
        arrowprops=dict(
            arrowstyle='<->',
            color=battery_color["orange"],
            linewidth=2,
        )
    )
    ax.text(
        mid_aged, 2.05,
        f'SoH = {soh*100:.0f}%',
        ha='center', va='bottom',
        color=battery_color["orange"]
    )

    plt.show()

interact(
    plot_aging_modes,
    LAMPE=FloatSlider(min=0, max=0.5, step=0.01, value=0, description="LAMPE"),
    LAMNE=FloatSlider(min=0, max=0.5, step=0.01, value=0, description="LAMNE"),
    LLI=FloatSlider(min=0, max=0.5, step=0.01, value=0, description="LLI")
)

interactive(children=(FloatSlider(value=0.0, description='LAMPE', max=0.5, step=0.01), FloatSlider(value=0.0, …

<function __main__.plot_aging_modes(LAMPE, LAMNE, LLI)>

In [60]:
soc_cell_aged_clipped

array([0.        , 0.00724334, 0.01750956, 0.02777579, 0.03804201,
       0.04830823, 0.05857446, 0.06884068, 0.0791069 , 0.08937313,
       0.09963935, 0.10990557, 0.1201718 , 0.13043802, 0.14070424,
       0.15097047, 0.16123669, 0.17150291, 0.18176914, 0.19203536,
       0.20230158, 0.21256781, 0.22283403, 0.23310025, 0.24336648,
       0.2536327 , 0.26389892, 0.27416515, 0.28443137, 0.29469759,
       0.30496382, 0.31523004, 0.32549626, 0.33576249, 0.34602871,
       0.35629493, 0.36656116, 0.37682738, 0.3870936 , 0.39735983,
       0.40762605, 0.41789227, 0.4281585 , 0.43842472, 0.44869094,
       0.45895717, 0.46922339, 0.47948961, 0.48975584, 0.50002206,
       0.51028828, 0.52055451, 0.53082073, 0.54108695, 0.55135318,
       0.5616194 , 0.57188562, 0.58215185, 0.59241807, 0.60268429,
       0.61295052, 0.62321674, 0.63348296, 0.64374919, 0.65401541,
       0.66428163, 0.67454786, 0.68481408, 0.6950803 , 0.70534653,
       0.71561275, 0.72587897, 0.7361452 , 0.74641142, 0.75667

In [61]:
soc_cell_clipped

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

np.float64(0.013333225355337763)